In [1]:
from reader import read, read_url
from grouper import RegionProfile
from color_maker import ColorMaker
from ipywidgets import widgets
from io import BytesIO
from zipfile import ZipFile
import plotly.graph_objects as go
import requests

rec_url = "https://portal.bluebrain.epfl.ch/wp-content/uploads/2019/02/white_matter_FULL_RECIPE_v1p15.yaml_-3.zip"
rec_fn = "white_matter_FULL_RECIPE_v1p15.yaml"

data, layer_labels = read_url(rec_url, rec_fn)
default_target_region = "SSp-ll"
default_grouping = ["Region", "Projection class", "None"]
cols = ColorMaker.factory()

r = [RegionProfile(data[default_target_region], layer_labels, colors=cols)]
valid_grouping = list(r[0]._types_of_groups.keys()) + ["None"]

In [2]:
layer_selector = widgets.Dropdown(description='Target Layer', value=layer_labels[0], options=layer_labels)
region_selector = widgets.Dropdown(description='Target Region', value=default_target_region, options=sorted(data.keys()))
grp1_selector = widgets.Dropdown(description='First grouping', value=default_grouping[0],
                                options=valid_grouping)
grp2_selector = widgets.Dropdown(description='Second grouping', value=default_grouping[1],
                                options=valid_grouping)
grp3_selector = widgets.Dropdown(description='Third grouping', value=default_grouping[2],
                                options=valid_grouping)
thresh_selector = widgets.FloatSlider(description="Threshold (*1E-3/um^3)", min=0.0, max=100, value=0.0, step=1.0)

r[0]._set_up_groups(order=[_x for _x in default_grouping if _x != "None"])
fig = r[0].plot_layer(0)

def full_update():
    l_idx = layer_labels.index(layer_selector.value)
    new_labels, offset, lbl_colors = r[0].get_plotly_labels()
    link_dict = r[0].get_plotly_links(offset, lbl_colors, l_idx, threshold=thresh_selector.value/1000)
    with fig.batch_update():
        fig.data[0].node.label = new_labels
        fig.data[0].node.color = lbl_colors
        fig.data[0].node.update()
        fig.data[0].link.update(link_dict)
    fig.update_layout()

def update_links(change):
    l_idx = layer_labels.index(layer_selector.value)
    _, offset, lbl_colors = r[0].get_plotly_labels()
    link_dict = r[0].get_plotly_links(offset, lbl_colors, l_idx, threshold=thresh_selector.value/1000)
    with fig.batch_update():
        fig.data[0].link.update(link_dict)

def update_region(change):
    r.pop()
    r.append(RegionProfile(data[region_selector.value], layer_labels, colors=cols))
    update_grouping(change)

def update_grouping(change):
    new_grouping = [grp1_selector.value, grp2_selector.value, grp3_selector.value]
    r[0]._set_up_groups(order=[_x for _x in new_grouping if _x != "None"])
    full_update()
    

layer_selector.observe(update_links, names="value")
thresh_selector.observe(update_links, names="value")
region_selector.observe(update_region, names="value")
grp1_selector.observe(update_grouping, names="value")
grp2_selector.observe(update_grouping, names="value")
grp3_selector.observe(update_grouping, names="value")
grp_box = widgets.HBox([grp1_selector, grp2_selector, grp3_selector])
widgets.VBox([widgets.HBox([layer_selector, region_selector, thresh_selector]),
              grp_box,
              fig])

# fig.show()
                              